In [1]:
import seaborn as sns
import pandas as pd
pd.set_option('display.max_colwidth',None)

In [2]:
from pathlib import Path

In [47]:
import eurostat

def load_df_raw_catalog():
    return eurostat.get_toc_df(lang = "en")

In [324]:
import re

from pandas.core.interchange.dataframe_protocol import DataFrame


def split_domain(code):
    match = re.match(r"^([A-Z]+?)([AQ])+(_)+(.*)",code)
    if match:
        return  match.group(1), match.group(2), match.group(4)
    return None, None, None

def convert_to_datetime(series):
    cleaned = series.astype('str').str.replace(r'([+-]\d{2}):(\d{2})', r'\1:\2', regex=True)
    return pd.to_datetime(cleaned, errors='coerce', utc=True)

def convert_object_to_category(df:DataFrame, threshold :int=5):
    description = df[df.select_dtypes('object')].describe()
    unique_counts = description.loc['unique']
    for column in unique_counts[unique_counts < threshold].index.tolist():
        df[column] = df[column].astype('category')
    return df

# Load and explore data

In [369]:
df_ds = load_df_raw_catalog()

In [370]:
df_ds.shape

(8031, 7)

In [371]:
df_ds.head()

,title,code,type,last update of data,last table structure change,data start,data end
0,Percentage of persons employed part-time by household composition,LFST_HHPTETY,dataset,2025-09-09T23:00:00+0200,2025-09-09T23:00:00+0200,2006,2024
1,Percentage of self-employed adults by number of children and age of youngest child,LFST_HHSECHI,dataset,2025-09-12T11:00:00+0200,2025-04-15T11:00:00+0200,2006,2024
2,Percentage of self-employed persons by household composition,LFST_HHSETY,dataset,2025-09-09T23:00:00+0200,2025-09-09T23:00:00+0200,2006,2024
3,Percentage of adult temporary employees by number of children and age of youngest child,LFST_HHTEMCHI,dataset,2025-09-12T11:00:00+0200,2025-04-15T11:00:00+0200,2006,2024
4,Percentage of temporary employees by household composition,LFST_HHTEMTY,dataset,2025-09-09T23:00:00+0200,2025-09-09T23:00:00+0200,2006,2024


In [372]:
df_ds.dtypes

title                          object
code                           object
type                           object
last update of data            object
last table structure change    object
data start                     object
data end                       object
dtype: object

In [373]:
df_ds.describe()

,title,code,type,last update of data,last table structure change,data start,data end
count,8031,8031,8031,8031,8031,7487,7487
unique,7497,8031,1,855,491,195,105
top,Individuals - internet activities,LFST_HHPTETY,dataset,2025-09-25T23:00:00+0200,2024-01-03T23:00:00+0100,2005,2024
freq,6,1,8031,688,1489,580,1891


### Remove not meaningful data

In [330]:
df_ds = df_ds.drop(columns='type')

## Set column data types

In [378]:
df_ds['title'] = df_ds['title'].astype('str')
df_ds['code'] = df_ds['code'].astype('str')
df_ds['last update of data'] = convert_to_datetime(df_ds['last update of data'])
df_ds['last table structure change'] = convert_to_datetime(df_ds['last table structure change'])

In [379]:
# official doc about values https://eurostat.github.io/restatapi/reference/get_eurostat_toc.html

In [398]:
df_ds['start_year'] = pd.to_numeric(df_ds['data start'], errors='coerce')
df_ds['end_year'] = pd.to_numeric(df_ds['data end'], errors='coerce')
df_ds = df_ds.drop(columns=['data start', 'data end'])

KeyError: 'data start'

In [381]:
df_ds.describe(include = 'all')

,title,code,type,last update of data,last table structure change,start_year,end_year
count,8031,8031,8031,8031,8031,6763.000000,6657.000000
unique,7497,8031,1,NaN,NaN,NaN,NaN
top,Individuals - internet activities,LFST_HHPTETY,dataset,NaN,NaN,NaN,NaN
freq,6,1,8031,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,2022-10-10 17:10:18.638401024+00:00,2024-10-15 00:40:18.862532608+00:00,2006.048647,2018.877873
min,NaN,NaN,NaN,2009-03-26 10:00:00+00:00,2018-12-13 12:00:00+00:00,1949.000000,1983.000000
25%,NaN,NaN,NaN,2022-02-04 22:00:00+00:00,2024-01-03 22:00:00+00:00,2000.000000,2015.000000
50%,NaN,NaN,NaN,2025-04-15 21:00:00+00:00,2025-01-27 22:00:00+00:00,2008.000000,2022.000000
75%,NaN,NaN,NaN,2025-09-11 21:00:00+00:00,2025-05-21 21:00:00+00:00,2014.000000,2024.000000
max,NaN,NaN,NaN,2025-09-29 21:00:00+00:00,2025-09-29 09:00:00+00:00,2024.000000,2100.000000


## Dataset cleanup  and relevance filtring

In [382]:
most_frequent = df_ds["title"].value_counts().reset_index()
titles = most_frequent.loc[most_frequent["count"]>1]["title"]
df_ds[df_ds['title'].isin(titles)].sort_values(by='title')

,title,code,type,last update of data,last table structure change,start_year,end_year
7399,"2014 - Number of employees by sex, economic activity and collective pay agreement",EARN_SES14_01$DV_351,dataset,2020-10-13 09:00:00+00:00,2024-06-07 21:00:00+00:00,NaN,NaN
7398,"2014 - Number of employees by sex, economic activity and collective pay agreement",EARN_SES14_01,dataset,2020-10-13 09:00:00+00:00,2024-06-07 21:00:00+00:00,2014.0,2014.0
7513,"2018 - Number of employees by sex, economic activity and collective pay agreement",EARN_SES18_01,dataset,2021-08-04 21:00:00+00:00,2024-01-03 22:00:00+00:00,2018.0,2018.0
7515,"2018 - Number of employees by sex, economic activity and collective pay agreement",EARN_SES18_01$DV_762,dataset,2021-08-04 21:00:00+00:00,2024-01-03 22:00:00+00:00,NaN,NaN
2920,Activities via internet not done because of security concerns (2010-2019),ISOC_CISCI_AX,dataset,2024-06-16 21:00:00+00:00,2024-01-03 22:00:00+00:00,2010.0,2019.0
...,...,...,...,...,...,...,...
4382,Young persons neither in employment nor in education and training by country of birth (NEET rates),EDAT_LFSE_28$DV_1849,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,NaN,NaN
4380,Young persons neither in employment nor in education and training by country of birth (NEET rates),EDAT_LFSE_28,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,2004.0,2024.0
4360,Young persons neither in employment nor in education and training by labour status (NEET rates),EDAT_LFSE_20$DV_1101,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,NaN,NaN
4362,Young persons neither in employment nor in education and training by labour status (NEET rates),EDAT_LFSE_20$DV_1807,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,NaN,NaN


It looks like datasets with duplicates titles include at least one version that has a $DV suffix followed by number.
This indicates a derived veiw (or data variation), which is not relevant for the purpose of this project. 

Therefore, I will exclude these derived view from the list of available datasets.


In [383]:
df_ds[df_ds['code'].str.contains('\\$DV', na = False)]

,title,code,type,last update of data,last table structure change,start_year,end_year
6,Percentage of employed adults working from home by number of children and age of youngest child,LFST_HHWAHCHI$DV_1872,dataset,2025-09-12 09:00:00+00:00,2025-04-15 09:00:00+00:00,NaN,NaN
42,"Employment rates by educational attainment level, country of birth and degree of urbanisation",LFST_R_EREDCOBU$DV_617,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,NaN,NaN
311,"Current depressive symptoms by sex, age and country of birth",HLTH_EHIS_MH1B$DV_2106,dataset,2023-06-16 21:00:00+00:00,2024-01-03 22:00:00+00:00,NaN,NaN
315,"Current depressive symptoms by sex, age and country of citizenship",HLTH_EHIS_MH1C$DV_2107,dataset,2023-06-16 21:00:00+00:00,2024-01-03 22:00:00+00:00,NaN,NaN
320,"Current depressive symptoms by sex, age and educational attainment level",HLTH_EHIS_MH1E$DV_463,dataset,2023-06-16 21:00:00+00:00,2024-01-03 22:00:00+00:00,NaN,NaN
...,...,...,...,...,...,...,...
7712,Employer business demography by size class and NACE Rev. 2 activity,BD_SALGE1_SIZE$DV_2306,dataset,2025-09-23 09:00:00+00:00,2025-09-23 09:00:00+00:00,NaN,NaN
7715,Business demography by size class and NACE Rev. 2 activity,BD_SIZE$DV_1483,dataset,2025-09-12 21:00:00+00:00,2025-09-10 21:00:00+00:00,NaN,NaN
7716,Business demography by size class and NACE Rev. 2 activity,BD_SIZE$DV_1502,dataset,2025-09-12 21:00:00+00:00,2025-09-10 21:00:00+00:00,NaN,NaN
7717,Business demography by size class and NACE Rev. 2 activity,BD_SIZE$DV_2304,dataset,2025-09-12 21:00:00+00:00,2025-09-10 21:00:00+00:00,NaN,NaN


In [384]:
df_ds = df_ds[~df_ds['code'].str.contains('\\$DV', na = False)]

Check for other duplicates titles

In [385]:
df_temp = df_ds.groupby(['title','start_year'])['code'].count().reset_index()
df_ds.loc[(df_ds["title"].isin(df_temp.loc[df_temp["code"]>1]['title'])&(df_ds["start_year"].isin(df_temp.loc[df_temp["code"]>1]['start_year'])))].sort_values(by='title')

,title,code,type,last update of data,last table structure change,start_year,end_year
2097,Aggregate replacement ratio for pensions (excluding other social benefits) by sex,ILC_PNP3,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2010.0,2024.0
3856,Aggregate replacement ratio for pensions (excluding other social benefits) by sex,TESPN070,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2010.0,2024.0
3952,At-risk-of-poverty rate before social transfers by sex - EU-SILC survey,TESSI230,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2013.0,2024.0
6181,At-risk-of-poverty rate before social transfers by sex - EU-SILC survey,TESOV250,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2013.0,2024.0
7437,Circular material use rate,SDG_12_41,dataset,2024-11-13 10:00:00+00:00,2025-08-08 21:00:00+00:00,2004.0,2023.0
5829,Circular material use rate,ENV_AC_CUR,dataset,2024-11-13 10:00:00+00:00,2025-08-08 21:00:00+00:00,2004.0,2023.0
7805,Circular material use rate,CEI_SRM030,dataset,2024-11-13 10:00:00+00:00,2025-08-08 21:00:00+00:00,2004.0,2023.0
7306,Early leavers from education and training by citizenship,SDG_04_10A,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,2004.0,2024.0
4326,Early leavers from education and training by citizenship,EDAT_LFSE_01,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,2004.0,2024.0
3622,Labour transitions from temporary to permanent contracts by sex -3-year average,ILC_LVHL36,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2008.0,2024.0


There are two types of naming:
1. {domain}{period}_{specific area} as an example Gross domestic product (GDP) and main components (output, expenditure and income) = NAM
    1. NAM*A*_10_GDP =  A for *Annual*
    2. NAMQ_10_GDP = Q for *Quater*
3. {domain}{number}, as example TIPSII40 = International investment position (tipsii) + number

In [388]:
df_ds[['ds_name','period','domain']] = df_ds['code'].apply(split_domain).apply(pd.Series)

In [389]:
# mark as quater ds that have start quater
df_ds['period'] = df_ds.apply(lambda row: 'A' if pd.isna(row.period) else 'Q',axis=1)

In [390]:
df_ds['period'] = df_ds['period'].astype('category')
df_ds.loc[(df_ds["title"].isin(df_temp.loc[df_temp["code"]>1]['title'])&(df_ds["start_year"].isin(df_temp.loc[df_temp["code"]>1]['start_year'])))].sort_values(by='title')

,title,code,type,last update of data,last table structure change,start_year,end_year,ds_name,period,domain
2097,Aggregate replacement ratio for pensions (excluding other social benefits) by sex,ILC_PNP3,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2010.0,2024.0,None,A,None
3856,Aggregate replacement ratio for pensions (excluding other social benefits) by sex,TESPN070,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2010.0,2024.0,None,A,None
3952,At-risk-of-poverty rate before social transfers by sex - EU-SILC survey,TESSI230,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2013.0,2024.0,None,A,None
6181,At-risk-of-poverty rate before social transfers by sex - EU-SILC survey,TESOV250,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2013.0,2024.0,None,A,None
7437,Circular material use rate,SDG_12_41,dataset,2024-11-13 10:00:00+00:00,2025-08-08 21:00:00+00:00,2004.0,2023.0,None,A,None
5829,Circular material use rate,ENV_AC_CUR,dataset,2024-11-13 10:00:00+00:00,2025-08-08 21:00:00+00:00,2004.0,2023.0,None,A,None
7805,Circular material use rate,CEI_SRM030,dataset,2024-11-13 10:00:00+00:00,2025-08-08 21:00:00+00:00,2004.0,2023.0,None,A,None
7306,Early leavers from education and training by citizenship,SDG_04_10A,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,2004.0,2024.0,None,A,None
4326,Early leavers from education and training by citizenship,EDAT_LFSE_01,dataset,2025-09-11 21:00:00+00:00,2025-04-14 21:00:00+00:00,2004.0,2024.0,None,A,None
3622,Labour transitions from temporary to permanent contracts by sex -3-year average,ILC_LVHL36,dataset,2025-09-25 21:00:00+00:00,2025-01-27 22:00:00+00:00,2008.0,2024.0,None,A,None


Duplicate dataset titles are still present, indicating that further analysis is needed to fully resolve and filter out all redundant entries.
So code is going to be used as a key.

In [391]:
df_ds.describe(include='all')

,title,code,type,last update of data,last table structure change,start_year,end_year,ds_name,period,domain
count,7580,7580,7580,7580,7580,6763.000000,6657.000000,411,7580,411
unique,7497,7580,1,NaN,NaN,NaN,NaN,8,2,313
top,House price index (2015 = 100) - quarterly data,LFST_HHPTETY,dataset,NaN,NaN,NaN,NaN,LFS,A,10_F_BS
freq,3,1,7580,NaN,NaN,NaN,NaN,212,7169,4
mean,NaN,NaN,NaN,2022-09-01 18:10:34.549472256+00:00,2024-10-16 02:35:03.520448512+00:00,2006.048647,2018.877873,NaN,NaN,NaN
min,NaN,NaN,NaN,2009-03-26 10:00:00+00:00,2018-12-13 12:00:00+00:00,1949.000000,1983.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,2021-11-18 16:00:00+00:00,2024-01-03 22:00:00+00:00,2000.000000,2015.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,2025-04-15 21:00:00+00:00,2025-01-27 22:00:00+00:00,2008.000000,2022.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,2025-09-11 21:00:00+00:00,2025-05-23 03:00:00+00:00,2014.000000,2024.000000,NaN,NaN,NaN
max,NaN,NaN,NaN,2025-09-29 21:00:00+00:00,2025-09-29 09:00:00+00:00,2024.000000,2100.000000,NaN,NaN,NaN


# Check na values

In [392]:
(df_ds.isna().sum()/len(df_ds))*100

title                           0.000000
code                            0.000000
type                            0.000000
last update of data             0.000000
last table structure change     0.000000
start_year                     10.778364
end_year                       12.176781
ds_name                        94.577836
period                          0.000000
domain                         94.577836
dtype: float64

In [393]:
df_ds.isna().value_counts()

title  code   type   last update of data  last table structure change  start_year  end_year  ds_name  period  domain
False  False  False  False                False                        False       False     True     False   True      6444
                                                                       True        True      True     False   True       717
                                                                       False       False     False    False   False      213
                                                                       True        True      False    False   False      100
                                                                       False       True      False    False   False       98
                                                                                             True     False   True         8
Name: count, dtype: int64

In [394]:
# domain and ds_name does not look useful, because most datasets are not follow that conversation

In [395]:
df_ds = df_ds.drop(columns=["ds_name","domain"])

In [396]:
df_ds.isna().value_counts()

title  code   type   last update of data  last table structure change  start_year  end_year  period
False  False  False  False                False                        False       False     False     6657
                                                                       True        True      False      817
                                                                       False       True      False      106
Name: count, dtype: int64

In [348]:
# then not all datasets have start year
df_ds[df_ds["start_year"].isna()]
# ALl of them has no dates 
df_ds[(df_ds["start_year"].isna())&(df_ds["end_year"].isna())]
# so they derived from original source with lates data 
# for example, env_wat_ltaa is ENV_WAT_RES for current year
# Change period type for them = ytd ~ year to date
df_ds["period"] = df_ds.apply(lambda x: 'ytd' if pd.isna(x["start_year"]) else x["period"], axis=1)
df_ds["start_year"] = df_ds.apply(lambda x: x["last update of data"].year, axis=1)
df_ds["end_year"] = df_ds.apply(lambda x: x["last update of data"].year, axis=1)

In [349]:
df_ds.isna().value_counts()

title  code   last update of data  last table structure change  start_year  start_quater  end_year  end_quater  period
False  False  False                False                        False       True          False     True        False     6750
                                                                            False         False     False       False      724
                                                                            True          False     False       False      106
Name: count, dtype: int64

In [353]:
all_without_start_period = df_ds[(df_ds["start_quater"].isna())].count().iloc[0]
all_without_start_period_annual = df_ds[(df_ds["start_quater"].isna())&(df_ds['period']=="A")].count().iloc[0]
if all_without_start_period == all_without_start_period_annual:
    print(f"All missing quaters are because of the annual statistics")
else:
    print(f"Only {all_without_start_period_annual} of {all_without_start_period} missed start period because of annual statistics")

Only 6763 of 6856 missed start period because of annual statistics


In [356]:
# all of them ytd?
all_without_start_period_ytd = df_ds[(df_ds["start_quater"].isna())&(df_ds['period']=="ytd")].count().iloc[0]
if all_without_start_period == all_without_start_period_annual + all_without_start_period_ytd:
    print(f"All missing quaters are because of the annual statistics")


All missing quaters are because of the annual statistics


In [357]:
df_ds["start_quater"] = df_ds["start_quater"].fillna('Q1')

In [ ]:
# so simular quation for ytd

In [363]:
df_ds["end_quater"] = df_ds.apply(lambda row: 'Q4' if row["period"]=='A' else "Q" + str(row["last update of data"].quarter) , axis =1)

In [364]:
df_ds.isna().value_counts()

title  code   last update of data  last table structure change  start_year  start_quater  end_year  end_quater  period
False  False  False                False                        False       False         False     False       False     7580
Name: count, dtype: int64

In [366]:
df_ds[(df_ds['period']=="ytd")]

,title,code,last update of data,last table structure change,start_year,start_quater,end_year,end_quater,period
5986,Water resources: long-term annual average,TEN00001,2025-08-01 21:00:00+00:00,2025-07-30 21:00:00+00:00,2025,Q1,2025,Q3,ytd
6097,Renewable freshwater resources - long term annual averages,ENV_WAT_LTAA,2025-08-01 21:00:00+00:00,2025-07-30 21:00:00+00:00,2025,Q1,2025,Q3,ytd
7929,International trade of EU and non-EU countries since 2002 by SITC,DS-059331,2025-09-15 09:00:00+00:00,2025-09-15 09:00:00+00:00,2025,Q1,2025,Q3,ytd
7930,Total production,DS-056121,2025-07-24 09:00:00+00:00,2025-07-24 09:00:00+00:00,2025,Q1,2025,Q3,ytd
7931,EU trade since 2017 by BEC/rev.5,DS-059329,2025-09-15 09:00:00+00:00,2025-09-15 09:00:00+00:00,2025,Q1,2025,Q3,ytd
...,...,...,...,...,...,...,...,...,...
8025,Prices of letter mail and parcel services (USP under direct or indirect designation),POST_CUBE1_X$POST_PRI_1,2025-04-15 12:44:56+00:00,2025-04-15 12:44:56+00:00,2025,Q1,2025,Q2,ytd
8026,"Postal services falling under the universal service obligation (USP under direct or indirect designation, traffic)",POST_CUBE1_X$USO701,2025-04-15 12:44:56+00:00,2025-04-15 12:44:56+00:00,2025,Q1,2025,Q2,ytd
8028,Number of enterprises providing postal services,POST_CUBE1_X$NUM701,2025-04-15 12:44:56+00:00,2025-04-15 12:44:56+00:00,2025,Q1,2025,Q2,ytd
8029,Access points (USP under direct or indirect designation ),POST_CUBE1_X$POST_ACC_1,2025-04-15 12:44:56+00:00,2025-04-15 12:44:56+00:00,2025,Q1,2025,Q2,ytd


## Final transformations

In [421]:
df = load_df_raw_catalog()

In [422]:
df.head()

,title,code,type,last update of data,last table structure change,data start,data end
0,Percentage of persons employed part-time by household composition,LFST_HHPTETY,dataset,2025-09-09T23:00:00+0200,2025-09-09T23:00:00+0200,2006,2024
1,Percentage of self-employed adults by number of children and age of youngest child,LFST_HHSECHI,dataset,2025-09-12T11:00:00+0200,2025-04-15T11:00:00+0200,2006,2024
2,Percentage of self-employed persons by household composition,LFST_HHSETY,dataset,2025-09-09T23:00:00+0200,2025-09-09T23:00:00+0200,2006,2024
3,Percentage of adult temporary employees by number of children and age of youngest child,LFST_HHTEMCHI,dataset,2025-09-12T11:00:00+0200,2025-04-15T11:00:00+0200,2006,2024
4,Percentage of temporary employees by household composition,LFST_HHTEMTY,dataset,2025-09-09T23:00:00+0200,2025-09-09T23:00:00+0200,2006,2024


In [423]:
df['title'] = df['title'].astype('str')
df['code'] = df['code'].astype('str')
df['last_update_timpestamp'] = convert_to_datetime(df['last update of data'])
df['last_structure_change_timpestamp'] = convert_to_datetime(df['last table structure change'])
df['start_year'] = pd.to_numeric(df['data start'], errors='coerce')
df['end_year'] = pd.to_numeric(df['data end'], errors='coerce')
df = df.drop(columns=['data start', 'data end','last update of data','last table structure change'])

In [424]:
df = df[~df['code'].str.contains('\\$DV', na = False)]

In [462]:
def get_period(code):
    match = re.match(r"^([A-Z]+?)([AQ])+(_)+(.*)",code)
    if match:
        return  match.group(2)
    match2 = re.match(r"^([A-Z_0-9])+(_)+([AQ])",code)
    if match2:
        return match2.group(3)
    return None

In [464]:
get_period("BOP_GDP6_A")

'A'

In [465]:
df['period'] = df["code"].apply(get_period)

In [466]:
df["period"] = df.apply(lambda x: 'ytd' if pd.isna(x["start_year"]) else x["period"], axis=1)
df["start_year"] = df.apply(lambda x: x["last_update_timpestamp"].year, axis=1)
df["end_year"] = df.apply(lambda x: x["last_update_timpestamp"].year, axis=1)

In [467]:
df.isna().value_counts()

title  code   type   last_update_timpestamp  last_structure_change_timpestamp  start_year  end_year  period
False  False  False  False                   False                             False       False     True      6259
                                                                                                     False     1321
Name: count, dtype: int64